<a href="https://colab.research.google.com/github/CBravoR/AdvancedAnalyticsLabs/blob/master/notebooks/python/Lab_PD_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PD Calibration

In this lab we will learn how to estimate the long-run PD after a model has been trained. The PD calibration can be done with the score, the monthly portfolio each case belongs to (usually the behavioural scorecard is used), the labels (Default / Non-Default) and a set of economic factors. For this work we will use an exchange rate and a commodity price.

First, we load the data. It is in Excel, so we use the appropriate function from Pandas. There are two worksheets: The first one contains the data for each borrower and each portfolio, and the second one contains the macro factor at each month.

In [ ]:
# Package load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Bigger and prettier plots
plt.rcParams['figure.figsize'] = (10, 5)
plt.style.use('fivethirtyeight')

In [ ]:
# Download Excel file
!gdown 'https://drive.google.com/uc?id=1UYmgsu5gI5U_VbraKXHxWTXyZSbM6q5S'

In [ ]:
# Load the data. Two datasets are necessary.
loans = pd.read_excel('PDCalExample.xlsx', # Filename
                      sheet_name=0,         # Worksheet name or index
                      )

econ_factors = pd.read_excel('PDCalExample.xlsx', # Filename
                             sheet_name=1,         # Worksheet name or index
                             )

In [ ]:
econ_factors

In [ ]:
loans

In [ ]:
loans.Portfolio.value_counts()

In [ ]:
loans.describe()

In [ ]:
econ_factors.describe()

Let's normalize the economic factors using a z-transform.

In [ ]:
from scipy.stats import zscore
econ_factors=econ_factors.apply(zscore)
econ_factors.describe()

## Defining Ratings

We have all the data we need. Let's start then by obtaining PD segments. Basel suggests building 7-15 segments. For this, we can use the excellent package [pwlf](https://pypi.org/project/pwlf/). It will allow segmenting a curve using a given number of cuts.

Which curve do we need to segment? The ROC curve!

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Calculate the ROC curve points
fpr, tpr, thresholds = roc_curve(loans['Default'],
                                 loans['Probs'])

# Save the AUC in a variable to display it. Round it first
auc = np.round(roc_auc_score(y_true = loans['Default'],
                             y_score = loans['Probs']),
               decimals = 3)

# Create and show the plot
plt.plot(fpr,tpr,label="Scorecard, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Install package
!pip install pwlf

Now we can segment the curve. The process takes a while to run, and sadly it is sequential, so go make yourself a coffee / tea while this runs.

In [ ]:
import pwlf

# Define the curve with the ROC curve
piecewise_AUC = pwlf.PiecewiseLinFit(fpr, tpr)

In [ ]:
# Calculate the best curve. Long!
res = piecewise_AUC.fit(10)

As this is a long process, it is a good idea to save the results. The object res can be pickled, or simply the cuts can be saved in a commented line.

In [ ]:
res

In [ ]:
# Use previous result
res = [0.        , 0.01383389, 0.03920524, 0.07731607, 0.11285577,
      0.20653665, 0.32339978, 0.41818781, 0.57283343, 0.7999917 ,
      1.
      ]

In [ ]:
ROC_curve = pd.DataFrame({'fpr': fpr, 'threshold': thresholds})
ROC_curve

To apply the cuts, you can use the method ```fit_with_breaks``` that is available for the ```piecewise_AUC``` object.


In [ ]:
# Apply cuts!
cuts = piecewise_AUC.fit_with_breaks(res)

We can now apply this to our dataset and see how the piecewise curve fits the ROC curve.

In [ ]:
# predict for the determined points
xHat = np.linspace(min(fpr), max(fpr), num=10000)
yHat = piecewise_AUC.predict(xHat)

# plot the results
plt.figure()
plt.plot(fpr, tpr, 'o')
plt.plot(xHat, yHat, '-')
plt.show()

We have a pretty good fit! But we are still not done, we now need to understand if the cuts lead to monotonic PDs. This is the final constraint. For this, we first calculate the PD for the whole dataset (we will adjust this later for each portfolio).

In [ ]:
# Find probability associated with every cut
pbb_cuts = np.zeros_like(res)
i = 0

for fpr in res:
  temp = ROC_curve.loc[np.round(ROC_curve.fpr, 2) == np.round(fpr, 2), 'threshold']
  pbb_cuts[i] = np.mean(temp)
  i += 1

pbb_cuts = np.flip(pbb_cuts)

In [ ]:
pbb_cuts = np.append(pbb_cuts, 1)
pbb_cuts = np.insert(pbb_cuts, 0, 0)
pbb_cuts

In [ ]:
pd_cut = pd.cut(loans['Probs'], pbb_cuts)
pd_cut

And we study the output with a crosstab.

In [ ]:
# Create table with cases total.
PDs_Tab = pd.crosstab(pd_cut,
                      loans['Default'],
                      normalize = False)

# Calculate default rate.
print(PDs_Tab)
pd_final = PDs_Tab[1] / (PDs_Tab[0] + PDs_Tab[1])
pd_final

The PD is perfectly monotonous! We should however combine the first three cuts, as the first two cuts contain no defaulters, and the last one, that seems to have to few cases. After doing this, our cuts are reasonable and we can now proceed to calculate the PDs for every portfolio. For this, we need to calculate the average number of defaults for each portfolio, over the total number of cases that month.

In [ ]:
# Adjusted cuts
pbb_cuts = [0.00000000e+00, 2.28978031e-01, # Delete first cut
            3.43684010e-01, 4.29913660e-01, 5.55186700e-01, 6.94758101e-01,
            7.51302438e-01, 8.26511005e-01, 9.11724503e-01, 9.82918518e-01,
            1.00000000e+00]

# Add the PDCut variable to our dataframe
loans['PD_Cut'] = pd.cut(loans['Probs'], pbb_cuts)

# Create pivot table
PD_monthly = pd.pivot_table(loans,
                            values = 'Default',
                            index = 'Portfolio',
                            columns = 'PD_Cut',
                            aggfunc = np.mean
                            )

PD_monthly

Now we have calculated the PDs for all ratings! We ended up with one less than the Basel recommendation, but this is only because we have very few loans in this toy example. On larger portfolios you will easily reach the 7-15 range.

Let's plot how our ratings look like.

In [ ]:
PD_monthly.plot(subplots=True,
          layout=(2, 5),
          sharex=False,
          sharey=False,
          colormap='viridis',
         fontsize=8,
         legend=False,
         linewidth=0.2);
plt.tight_layout();

We see that some months have no defaulters. With the very limited data we have that is to be expected, but in large portfolios this will be softer. In general all time series look stationary.

And that's almost it! Now we are ready to calibrate these PDs across the multiple cuts.

## Estimating Long-Term PD

To estimate the long-term PD, we need to estimate what the average PD is given the last PD and the macroeconomic factors. For this, we can use the subpackage [Time Series Analysis](https://www.statsmodels.org/stable/tsa.html#module-statsmodels.tsa) (```tsa```) of the ```statsmodel``` package. We aim to run a SARIMAX model, so we should study stationary properties, seasonality, etc.

The first step is to turn the Portfolio index into a TimeSeries index. This way Python knows it is dealing with dates. Our data starts in January 1999 and it is monthly data. We arbitrarily set our first day as the 1st of January 1999 and add up from there.

In [ ]:
from datetime import date
from dateutil.relativedelta import *

start_date = date(1999, 1, 1)
PD_monthly.index = [pd.to_datetime(start_date + relativedelta(months=+portfolio_month-1)) for portfolio_month in PD_monthly.index]
econ_factors.index = [pd.to_datetime(start_date + relativedelta(months=+portfolio_month-1)) for portfolio_month in econ_factors.index]
econ_factors.drop(columns='Portfolio', inplace = True)

In [ ]:
PD_monthly.iloc[:, 4]

Now we can decompose our time series to see what is happening.



In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(PD_monthly.iloc[:, 4], model='additive')
fig = decomposition.plot()
plt.show()

The time series look fairly stationary, with yearly seasonality. We are read to estimate a model!

Within the ```tsa``` package, the [SARIMAX model](https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html#statsmodels.tsa.statespace.sarimax.SARIMAX), present in the subpackage ```statespace```, is the most general model available, allowing for Seasonality, Auto Regression, Integration, Moving Averages, and eXogenous factors. We will train a simpler ARX model (autoregressive with exogenous regressors), but you are welcome to experiment further, more complex, regressions.

Let's search for the best model for a rating, searching between 1 and 6 autoregression factors, using the macroeconomic factors as the exogenous variables.

In [ ]:
# Define the search space.
p = range(1, 6)
d = range(0, 2)
q = range(0, 2)

# Create an interative list of ps, ds, qs.
from itertools import product
pdq = list(product(p, d, q))

# Seasonal parameters. One year back.
ps = range(0, 4)
ds = range(0, 1)
qs = range(0, 1)
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(product(ps, ds, qs))]

# Train the models for a series and test multiple values.
y = PD_monthly.iloc[:, 4] # Choose the fifth rating

from statsmodels.tsa.statespace.sarimax import SARIMAX

auc_out = []

for param in pdq:
  for param_seasonal in seasonal_pdq:
      mod = SARIMAX(y,
                    exog=np.asarray(econ_factors),
                    order=param,
                    seasonal_order=param_seasonal,
                    enforce_stationarity=False,
                    enforce_invertibility=False
                    )
      results = mod.fit()
      auc_out.append([param, param_seasonal, results.aic])
      print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))

# Nicer formatting
auc_out = pd.DataFrame(auc_out,
                       columns = ['(p,q,r)', '(ps, qs, rs, S)', 'AIC'])

The warning is due to the frequency of the time series which is not provided by Pandas. You can safely ignore it.

Let's see which one is the resulting model. AIC should be minimized.

In [ ]:
auc_out.sort_values(by='AIC', ascending=True)


We can see from these values that the prefer model is an autoregressive of order 1, with 12 month seasonality.

Let's test the final model for these values.

In [ ]:
# ARIMA(1, 1, 1)x(0, 0, 0, 12)12
mod_BB = SARIMAX(y,
              exog=np.asarray(econ_factors),
              order=(1,1,1),
              seasonal_order=(0,0,0,12),
              enforce_stationarity=False,
              enforce_invertibility=False)
results_BB = mod_BB.fit()

print(results_BB.summary().tables[1])

In [ ]:
# ARIMA(2, 0, 1)x(0, 0, 0, 12)12
mod_BB = SARIMAX(y,
              exog=np.asarray(econ_factors),
              order=(2,0,1),
              seasonal_order=(0,0,0,12),
              enforce_stationarity=False,
              enforce_invertibility=False)
results_BB = mod_BB.fit()

print(results_BB.summary().tables[1])

We pick the one that **minimizes** the AIC, which is ARIMA(1, 0, 1)x(0, 0, 0, 12).

> Indented block



In [ ]:
# ARIMA(1, 0, 1)x(0, 0, 0, 12)12
mod_BB = SARIMAX(y,
              exog=np.asarray(econ_factors),
              order=(1,0,0),
              seasonal_order=(0,0,0,12),
              enforce_stationarity=False,
              enforce_invertibility=False)
results_BB = mod_BB.fit()

print(results_BB.summary().tables[1])

The results show what, for this dataset, there is not much exogenous information that is useful. This would mean we need to go look for better economic factors! Another interesting result is that the L1 and L2 parameters are not useful either, which means that it is only the third lag that carries the information.

It is not perfect, so we should be a bit careful with this model and possibly study more complex structures. I leave this as an exercise!

And that's it! Repeating this process for every time series leads to our calibrated PDs. Now what we would do is to use the long-run estimates for the economic factors to reach a long-term PD.

For LGD, the process is the same. The only difference is that you need to use the **downturn** economic factors instead of the long-run economic factors